# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession


In [3]:
spark=SparkSession.builder.getOrCreate()
df=spark.sql("select 'spark' as hello")
df.show()


+-----+
|hello|
+-----+
|spark|
+-----+



## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [4]:

df=spark.read.csv("D:\googleplaystore.csv",header=True,inferSchema=True)
df.show()
df.toPandas()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [5]:
df.show(5)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

As well as the schema to make sure all the column types were correctly infered

In [6]:
df.columns

['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver']

Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [7]:
from pyspark.sql.types import IntegerType, FloatType
newdf = df.withColumn("Rating", df["Rating"].cast(FloatType())) \
            .withColumn("Reviews", df["Reviews"].cast(IntegerType())) \
            .withColumn("Price", df["Price"].cast(IntegerType()))
print(newdf.printSchema())
newdf.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [8]:
df.createOrReplaceTempView("tempview")

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [9]:
r=spark.sql("select * from tempview where Rating>4.1")
r.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644| 25M|50,000,000+|Free|    0|          Teen|        Art & Design|      June 8, 2018|Varies with device|  4.2 and up|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|2.8M|   100,000+|Free|    0|      Everyone|Art & Design;Crea...|     June 20, 2018|               1.1|  4.4 

## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [53]:
r=spark.sql("select * from tempview where Rating>4.5 and Category='COMICS'")
r.show()

+---------------------+--------+------+-------+----+----------+----+-----+---------------+-----------------+-----------------+-----------+------------+
|                  App|Category|Rating|Reviews|Size|  Installs|Type|Price| Content Rating|           Genres|     Last Updated|Current Ver| Android Ver|
+---------------------+--------+------+-------+----+----------+----+-----+---------------+-----------------+-----------------+-----------+------------+
| Manga Master - Be...|  COMICS|   4.6|  24005|4.9M|  500,000+|Free|    0|Adults only 18+|           Comics|     July 4, 2018|    1.1.7.0|  4.1 and up|
| GANMA! - All orig...|  COMICS|   4.7|  57106| 27M|1,000,000+|Free|    0|           Teen|           Comics|    July 23, 2018|     2.11.3|  4.1 and up|
| Röhrich Werner So...|  COMICS|   4.7|   2249| 32M|  500,000+|Free|    0|       Everyone|           Comics|November 16, 2017|       1.08|4.0.3 and up|
| Unicorn Pokez - C...|  COMICS|   4.8|    516| 12M|   50,000+|Free|    0|       Everyon

## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [11]:
d=spark.sql('select category,sum(reviews)from tempview group by category')
d.show()

+-------------------+-------------+
|           category| sum(reviews)|
+-------------------+-------------+
|             EVENTS|     161018.0|
|             COMICS|    3383276.0|
|             SPORTS|  7.0830169E7|
|            WEATHER|  1.4604735E7|
|      VIDEO_PLAYERS| 1.10380188E8|
|  AUTO_AND_VEHICLES|    1163666.0|
|          PARENTING|     958331.0|
|      ENTERTAINMENT|  5.9178154E7|
|    PERSONALIZATION|   8.934614E7|
| HEALTH_AND_FITNESS|  3.7891234E7|
|   TRAVEL_AND_LOCAL|  6.2617919E7|
|BOOKS_AND_REFERENCE|  2.1959069E7|
|     FOOD_AND_DRINK|    8883330.0|
|        PHOTOGRAPHY|  2.1351665E8|
|           BUSINESS|  1.3954552E7|
|             FAMILY|  4.1022633E8|
|           SHOPPING| 1.15041222E8|
|     HOUSE_AND_HOME|    3976385.0|
|               GAME|1.585422349E9|
|               Face|         null|
+-------------------+-------------+
only showing top 20 rows



## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [12]:
res=spark.sql("select App,reviews from tempview order by reviews desc")
res.show()

+--------------------+-------------+
|                 App|      reviews|
+--------------------+-------------+
|"Women""s Health ...|weight lose)"|
|GollerCepte Live ...|         9992|
|Ad Block REMOVER ...|          999|
| SnipSnap Coupon App|         9975|
| SnipSnap Coupon App|         9975|
|US Open Tennis Ch...|         9971|
|US Open Tennis Ch...|         9971|
|          DreamTrips|         9971|
|Adult Color by Nu...|          997|
|BSPlayer ARMv7 VF...|         9966|
|Easy Resume Build...|          996|
|   MegaFon Dashboard|        99559|
|Buff Thun - Daily...|         9952|
|       Al jazeera TV|         9952|
|Rope Hero Return ...|         9951|
|Family Guy The Qu...|       995002|
|tutti.ch - Free C...|         9950|
|    Teach Me Anatomy|         9945|
|          Funny Pics|         9941|
| Best Car Wallpapers|          994|
+--------------------+-------------+
only showing top 20 rows



## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [51]:
r=spark.sql("select app from tempview where app like'%dating%'")
r.show()

+--------------------+
|                 app|
+--------------------+
|Meet, chat & date...|
|Friend Find: free...|
|Spine- The dating...|
|Princess Closet :...|
|happn – Local dat...|
+--------------------+



## 6. Use SQL Transformer to display how many free apps there are in this list

In [40]:
from pyspark.ml.feature import SQLTransformer
sqltrans=SQLTransformer(statement='select app from __THIS__ where Type="Free"')
sqltrans.transform(df).show()

+--------------------+
|                 app|
+--------------------+
|Photo Editor & Ca...|
| Coloring book moana|
|U Launcher Lite –...|
|Sketch - Draw & P...|
|Pixel Draw - Numb...|
|Paper flowers ins...|
|Smoke Effect Phot...|
|    Infinite Painter|
|Garden Coloring Book|
|Kids Paint Free -...|
|Text on Photo - F...|
|Name Art Photo Ed...|
|Tattoo Name On My...|
|Mandala Coloring ...|
|3D Color Pixel by...|
|Learn To Draw Kaw...|
|Photo Designer - ...|
|350 Diy Room Deco...|
|FlipaClip - Carto...|
|        ibis Paint X|
+--------------------+
only showing top 20 rows



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [47]:
#r=spark.sql("select Genres from tempview group by'Genres' order by count() desc")
#r.show()
r=spark.sql("SELECT Genres,count(Genres) as value_occurrence from Tempview group by Genres order by value_occurrence desc")
r.show(5)

+-------------+----------------+
|       Genres|value_occurrence|
+-------------+----------------+
|        Tools|             842|
|Entertainment|             623|
|    Education|             549|
|      Medical|             463|
|     Business|             460|
+-------------+----------------+
only showing top 5 rows



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [56]:
r=spark.sql("select app,genres from tempview where Reviews>100 and gENRES='Tools'")
r.show()

+--------------------+------+
|                 app|genres|
+--------------------+------+
|   Moto File Manager| Tools|
|              Google| Tools|
|    Google Translate| Tools|
|        Moto Display| Tools|
|      Motorola Alert| Tools|
|     Motorola Assist| Tools|
|Cache Cleaner-DU ...| Tools|
|  Moto Suggestions ™| Tools|
|          Moto Voice| Tools|
|          Calculator| Tools|
|         Device Help| Tools|
|     Account Manager| Tools|
|             myMetro| Tools|
|        File Manager| Tools|
|           My Telcel| Tools|
|Calculator - free...| Tools|
| ASUS Sound Recorder| Tools|
|  iWnn IME for Nexus| Tools|
|Samsung Max - Dat...| Tools|
|          ZenUI Help| Tools|
+--------------------+------+
only showing top 20 rows



## That's all folks! Great job!